In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import theano
import theano.tensor as T
import lasagne

import imageio
from skimage import transform

from glob import glob
from tqdm import tqdm

import os
import pickle
from queue import Queue
from random import randint, shuffle
import threading
from time import time
from itertools import chain

%matplotlib inline

In [2]:
from lasagne.layers import InputLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import BatchNormLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import DenseLayer
from lasagne.nonlinearities import rectify, softmax


def build_simple_block(incoming_layer, names,
                       num_filters, filter_size, stride, pad,
                       use_bias=False, nonlin=rectify):
    """Creates stacked Lasagne layers ConvLayer -> BN -> (ReLu)
    Parameters:
    ----------
    incoming_layer : instance of Lasagne layer
        Parent layer
    names : list of string
        Names of the layers in block
    num_filters : int
        Number of filters in convolution layer
    filter_size : int
        Size of filters in convolution layer
    stride : int
        Stride of convolution layer
    pad : int
        Padding of convolution layer
    use_bias : bool
        Whether to use bias in conlovution layer
    nonlin : function
        Nonlinearity type of Nonlinearity layer
    Returns
    -------
    tuple: (net, last_layer_name)
        net : dict
            Dictionary with stacked layers
        last_layer_name : string
            Last layer name
    """
    names = list(names)
    net = []
    net.append((
            names[0],
            ConvLayer(incoming_layer, num_filters, filter_size, stride, pad,
                      nonlinearity=None) if use_bias
            else ConvLayer(incoming_layer, num_filters, filter_size, stride, pad, b=None,
                           nonlinearity=None)
        ))

    net.append((
            names[1],
            BatchNormLayer(net[-1][1])
        ))
    if nonlin is not None:
        net.append((
            names[2],
            NonlinearityLayer(net[-1][1], nonlinearity=nonlin)
        ))

    return dict(net), net[-1][0]


def build_residual_block(incoming_layer, ratio_n_filter=1.0, ratio_size=1.0, has_left_branch=False,
                         upscale_factor=4, ix=''):
    """Creates two-branch residual block
    Parameters:
    ----------
    incoming_layer : instance of Lasagne layer
        Parent layer
    ratio_n_filter : float
        Scale factor of filter bank at the input of residual block
    ratio_size : float
        Scale factor of filter size
    has_left_branch : bool
        if True, then left branch contains simple block
    upscale_factor : float
        Scale factor of filter bank at the output of residual block
    ix : int
        Id of residual block
    Returns
    -------
    tuple: (net, last_layer_name)
        net : dict
            Dictionary with stacked layers
        last_layer_name : string
            Last layer name
    """
    simple_block_name_pattern = ['res%s_branch%i%s', 'bn%s_branch%i%s', 'res%s_branch%i%s_relu']

    net = {}

    # right branch
    net_tmp, last_layer_name = build_simple_block(
        incoming_layer, map(lambda s: s % (ix, 2, 'a'), simple_block_name_pattern),
        int(lasagne.layers.get_output_shape(incoming_layer)[1]*ratio_n_filter), 1, int(1.0/ratio_size), 0)
    net.update(net_tmp)

    net_tmp, last_layer_name = build_simple_block(
        net[last_layer_name], map(lambda s: s % (ix, 2, 'b'), simple_block_name_pattern),
        lasagne.layers.get_output_shape(net[last_layer_name])[1], 3, 1, 1)
    net.update(net_tmp)

    net_tmp, last_layer_name = build_simple_block(
        net[last_layer_name], map(lambda s: s % (ix, 2, 'c'), simple_block_name_pattern),
        lasagne.layers.get_output_shape(net[last_layer_name])[1]*upscale_factor, 1, 1, 0,
        nonlin=None)
    net.update(net_tmp)

    right_tail = net[last_layer_name]
    left_tail = incoming_layer

    # left branch
    if has_left_branch:
        net_tmp, last_layer_name = build_simple_block(
            incoming_layer, map(lambda s: s % (ix, 1, ''), simple_block_name_pattern),
            int(lasagne.layers.get_output_shape(incoming_layer)[1]*4*ratio_n_filter), 1, int(1.0/ratio_size), 0,
            nonlin=None)
        net.update(net_tmp)
        left_tail = net[last_layer_name]

    net['res%s' % ix] = ElemwiseSumLayer([left_tail, right_tail], coeffs=1)
    net['res%s_relu' % ix] = NonlinearityLayer(net['res%s' % ix], nonlinearity=rectify)

    return net, 'res%s_relu' % ix


def build_model():
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    sub_net, parent_layer_name = build_simple_block(
        net['input'], ['conv1', 'bn_conv1', 'conv1_relu'],
        64, 7, 2, 3, use_bias=True)
    net.update(sub_net)
    net['pool1'] = PoolLayer(net[parent_layer_name], pool_size=3, stride=2, pad=0, mode='max', ignore_border=False)
    block_size = list('abc')
    parent_layer_name = 'pool1'
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1, 1, True, 4, ix='2%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='2%s' % c)
        net.update(sub_net)

    block_size = list('abcd')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='3%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='3%s' % c)
        net.update(sub_net)

    block_size = list('abcdef')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='4%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='4%s' % c)
        net.update(sub_net)

    block_size = list('abc')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='5%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='5%s' % c)
        net.update(sub_net)
    net['pool5'] = PoolLayer(net[parent_layer_name], pool_size=7, stride=1, pad=0,
                             mode='average_exc_pad', ignore_border=False)
    net['fc1000'] = DenseLayer(net['pool5'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc1000'], nonlinearity=softmax)

    return net

In [3]:
model = build_model()
resnet = pickle.load(open('resnet50.pkl', 'rb'), encoding='latin1')
lasagne.layers.set_all_param_values(model['prob'], resnet['values'])

In [4]:
model1 = {}
model1['dense1'] = DenseLayer(model['pool5'], 500)
model1['dense2'] = DenseLayer(model1['dense1'], 500)
model1['final'] = DenseLayer(model1['dense2'], 101, nonlinearity=softmax)

In [5]:
all_params = []
for v in model1.values():
    all_params += v.get_params(trainable=True)
for k, v in model.items():
    if k.startswith('res5'):
        all_params += v.get_params(trainable=True)

In [6]:
net_in = T.tensor4()
net_trg = T.ivector()
net_out = T.matrix()

pred = lasagne.layers.get_output(model1['final'], net_in)
pred_det = lasagne.layers.get_output(model1['final'], net_in, deterministic=True)

acc = lasagne.objectives.categorical_accuracy(pred_det, net_trg).mean()
loss = lasagne.objectives.categorical_crossentropy(pred, net_trg).mean()

updates = lasagne.updates.adam(loss, all_params, learning_rate=0.0015)

In [19]:
train = theano.function([net_in, net_trg], loss, updates=updates)
val = theano.function([net_in, net_trg], acc)
pred = theano.function([net_in], pred_det)

In [12]:
def iter_data(dir_name, mask, frames_cnt=5, imsize=(200, 200), df=None):
    for file_name in glob(os.path.join(dir_name, mask)):
        name = os.path.split(file_name)[-1]
        if df is not None:
            name = df[df.filename==name].classnum.values[0]
        with imageio.get_reader(file_name) as reader:
            length = reader.get_length()
            for i in range(frames_cnt):
                frame = reader.get_data(randint(0, length - 1))
                frame = transform.resize(frame, imsize, mode='constant')
                frame = frame.swapaxes(2, 1).swapaxes(1, 0)
                yield (frame, name)

In [13]:
train_df = pd.read_csv('train_gt.csv')

In [23]:
iter_train = iter_data('action-recognition-train', '[0-7]*.avi', frames_cnt=1, df=train_df)
iter_val = iter_data('action-recognition-train', '8*.avi', frames_cnt=1, df=train_df)

In [17]:
def iter_batches(data, size):
    bx, by = [], []
    for x, y in data:
        bx.append(x)
        by.append(y)
        if len(bx) == size:
            yield (np.array(bx, dtype=np.float32), by)
            bx, by = [], []
    if len(by):
        yield (np.array(bx, dtype=np.float32), by)

In [24]:
for i in range(10):
    start = time()
    
    err, cnt = 0., 0
    for x, y in tqdm(iter_batches(iter_train, size=200)):
        err += train(x, y)
        cnt += 1
    train_err = err / cnt

    err, cnt = 0., 0
    for x, y in tqdm(iter_batches(iter_val, size=200)):
        err += val(bx, by)
        batches += 1
    val_err = err / cnt

    end = time()
    print('epoch: {} [time: {}] train loss: {}, val loss: {}'.format(i, end - start, train_err, val_err))



0it [00:00, ?it/s]

1it [02:45, 165.68s/it]

KeyboardInterrupt: 

In [25]:
FRAMES_CNT = 3
iter_test = iter_data('action-recognition-test', '*.avi', frames_cnt=FRAMES_CNT)

In [ ]:
results = {}
for x, y in tqdm(iter_batches(iter_test, size=200)):
    res = pred(x)
    res = res.reshape([-1, FRAMES_CNT, 101])
    res = res.mean(axis=1).argmax(axis=1)
    for file_name, res in zip(y[::FRAMES_CNT], res):
        results[file_name] = res

In [47]:
result_file = open('result.csv', 'w')
result_file.write('filename,classnum\n')
for file_name, class_label in results.items():
    result_file.write('{},{}\n'.format(file_name, class_label))
result_file.close()